# SRV releases

Owners: **PatriciaLarsen [@plarsen](https://github.com/LSSTDESC/DC2-analysis/issues/new?body=@patricialarsen)**  
Last Verifed to Run: **2024-06-04** (by @plarsen)

This notebook details the flags and cuts relevant to the SRV releases for the DP0.2 object catalog. We will discuss the quality flags provided by the Rubin Observatory, and extra ones we choose to apply.

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

### Release notes:

The choices for fiducial quality cuts for DESC key analyses will be described and justified below. Currently these are *draft* options for DP0.2, informed by precursor surveys, Rubin advice and our validations.  

In the long-term, working groups will likely want to apply slightly different quality cuts depending on their requirements. For instance, [Rau et. al](https://arxiv.org/abs/2406.11950v1) detail a method of significantly reducing the impact of photometric redshift error on cluster mass measurements by removing a small fraction of galaxies from the analysis. Our aim here is to validate and maintain the pre-selections used for key projects, and to give working groups a reasonable data pre-selection and the tools they need to understand how to iterate on this for specific analyses. 

 

First we'll detail the LSST flags that are applied. These flag descriptions are taken from the DP0.2 schema, found [here](https://dm.lsst.org/sdm_schemas/browser/dp02.html).


## Base LSST flags

The detect_isPrimary flag should always be applied to remove artificial objects and boundary duplicates, and the refExtendedness (extendedness in the reference band) should be applied to select for galaxies (or inversely stars). Finally we choose to remove objects which the deblender has skipped as we cannot be sure if they are blended objects.  

| Flag name | True/False  | Description   |
|:---|:-------------:|:-----------:|
| detect_isPrimary | True  | Removes duplicate detections (primarily from coadd edge regions) as well as skyobjects   |
| refExtendedness  | 1 (should be ne 0??)    | Source is classified as a galaxy | 
| deblend_skipped  | False | Removes objects skipped by deblender | 


### Pixel flags (center)

Pixel flags are then applied to remove "bad" pixels. Bad pixels are those for which 
- a cosmic ray may have passed through 
- the pixel is saturated
- the pixel is "suspect"
- the pixel may have been clipped
- the pixel is "interpolated"
(note to self: add explanations for all of these). 

The Rubin project provides flags for all of these, both for pixels near the source center and for the source footprint (wider region). We apply all these flags.

| Flag name | True/False  | Description   |
|:---|:-------------:|:-----------:|
| g_pixelFlags_clippedCenter | False  | Center is close to clipped pixel  |
| g_pixelFlags_crCenter | False  | Cosmic ray in source center   |
| g_pixelFlags_edge | False  | Source outside usable exposure region   |
| g_pixelFlags_inexact_psfCenter | False  | Source center close to inexact psf pixels   |
| g_pixelFlags_interpolatedCenter | False  | Interpolated pixel in source center   |
| g_pixelFlags_offimage | False  | Source center off image   |
| g_pixelFlags_saturatedCenter | False  | Saturated pixel in source center   |
| g_pixelFlags_sensor_edgeCenter | False  | Source center close to sensor edge pixel   |
| g_pixelFlags_suspectCenter | False  | Source center close to suspect pixel   |



### Pixel flags (footprint)

| Flag name | True/False  | Description   |
|:---|:-------------:|:-----------:|
| g_pixelFlags_bad | False  | Shows a bad pixel in the source footprint   |
| g_pixelFlags_clipped | False  | Footprint includes clipped pixels   |
| g_pixelFlags_cr | False  | Cosmic ray in footprint   |
| g_pixelFlags_inexact_psf | False  | Source includes inexact psf pixels   |
| g_pixelFlags_interpolated | False  | Interpolated pixel in source footprint   |
| g_pixelFlags_offimage | False  | Source center off image   |
| g_pixelFlags_saturated | False  | Saturated pixel in source footprint   |
| g_pixelFlags_sensor_edge | False  | Source footprint includes sensor edge pixel   |
| g_pixelFlags_suspect | False  | Source footprint includes suspect pixel   |


### Photometry flags

Flags are provided for failures of the photometry as follows.

We also note that there is plenty of information as to the accuracy of different flux measurements. For stars Rubin recommends use of the _psf_flux version, along with the _pixelFlags_inexact_psfCenter flag.

For galaxies, _cModelFlux is recommended for fluxes/magnitudes, and gaap fluxes for colors. They suggest the _gaap1p0Flux (1-arcsec aperture) option, as the optimal aperture seems to fail frequently (as noted in https://dp0-2.lsst.io/data-products-dp0-2/index.html)



| Flag | True/False  | Description   |
|:---|:-------------:|:-----------:|
| i_cModel_flag | False  | Failure of cModel fit   |
| i_gaapFlux_flag | False | Failure of gaap flux | 

### Extra flags

| Flag name | True/False  | Description   |
|:---|:-------------:|:-----------:|
| detect_isIsolated | True  | Removes blended objects |
| i_centroid_flag | False  | Failed centroid measurement |


### Photometry cuts


| Flag | Exact Cut  | Description   |
|:---|:-------------:|:-----------:|
| cmodel_magnitude | mag(i_cModel_flux)<24.5 *  | Limiting to match spectroscopic training and calibration data   |
| cmodel S/N       | cModel_S/N > 10, at least two g/r/z/y S/N > 5       | restricting unforced signal to noise      |
| blendedness      | i_blendedness < 10^(-0.375) | removing objects with strong neighbour contamination (not mildly blended objects) | 


*note: should be after extinction correction but this is unavailable for DP0.2


### Shear cuts

These are currently being applied to HSM shapes. Once a metadetect catalog is available these are expected to change substantially. These cuts are currently taken from [Mandelbaum et. al 2018](https://academic.oup.com/pasj/article/70/SP1/S25/4774314#431908315)



| Flag | Exact Cut  | Description   |
|:---|:-------------:|:-----------:|
| i_hsmShapeRegauss_sigma | 0 < val < 0.4 | Shape measurement error reasonable (and not NaN)  |
| i_hsmShapeRegauss_e | abs(val) < 2 | Absolute distortion cut   |
| i_hsmShapeRegauss_res | val > 0.3 | Absolute distortion cut   |
| iPSF_flag  | False |  PSF measurement flag | 
| i_flag     | False |  Shape measurement flag | 

where hsmShapeRegauss_res is defined in eq 4 of [Mandelbaum et. al 2018](https://academic.oup.com/pasj/article/70/SP1/S25/4774314#431908315) as $1 - T_{PSF}/T_{Image}$, where $T_{PSF} = \rm{i\_ixxPSF} + \rm{i\_iyyPSF}$ and $T_{Image} = \rm{i\_ixx} + \rm{i\_iyy}$

